In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import lines
import matplotlib.cm as cm
import tqdm
from tqdm.notebook import tqdm as tqdm

_cmap = plt.cm.jet

In [2]:
import multi_process as mp

In [3]:
k = mp.Foo()

In [4]:
k.re()

roo
